<a href="https://colab.research.google.com/github/Beyonder123/MovieReccomendSYS/blob/main/MovieReccomendSYS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Step 0: Install the necessary library
!pip install fuzzywuzzy[speedup]

In [6]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
from scipy.sparse import csr_matrix

!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# User-Movie Matrix
user_movie_matrix = ratings.pivot(index='movieId', columns='userId', values='rating')
user_movie_matrix_filled = user_movie_matrix.fillna(0)

matrix_sparse = csr_matrix(user_movie_matrix_filled.values)

# KNN model
cf_knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
cf_knn_model.fit(matrix_sparse)

def movie_recommender_engine(movie_name, matrix, cf_model, no_of_records):
    cf_model.fit(matrix)
    movie_id = process.extractOne(movie_name, movies['title'])[2]

    distances, indices = cf_model.kneighbors(matrix[movie_id], n_neighbors=no_of_records)

    movie_rec_ids = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]

    cf_recs = []
    for i in movie_rec_ids:
        cf_recs.append({'Title': movies['title'][i[0]], 'Distance': i[1]})

    df = pd.DataFrame(cf_recs, index=range(1, no_of_records))
    return df

no_of_records = 5
matrix = matrix_sparse
movie_name = "Jurassic Park"
recommendations = movie_recommender_engine(movie_name, matrix, cf_knn_model,no_of_records)

print(f"Recommendations for '{movie_name}':\n", recommendations)


--2024-09-28 15:52:37--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip.3’

ml-latest-small.zip 100%[===================>] 955.28K  3.18MB/s    in 0.3s    

2024-09-28 15:52:38 (3.18 MB/s) - ‘ml-latest-small.zip.3’ saved [978202/978202]

Archive:  ml-latest-small.zip
replace ml-latest-small/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-latest-small/links.csv  
replace ml-latest-small/tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-latest-small/tags.csv  
replace ml-latest-small/ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ml-latest-small/ratings.csv  
replace ml-latest-small/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflati